In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn.functional as F


In [2]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size,output_size):
        super(SimpleNN, self).__init__()
        self.output_size = output_size
        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.hidden(x)
        out = self.relu(out)
        out = self.output(out)
        return out


In [3]:
def initialize_population(population_size, num_genes):
    return np.random.randint(0, 2, size=(population_size, num_genes))

def decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range, num_genes):
    learning_rate = learning_rate_range[0] + int(''.join(map(str, chromosome[:num_genes//2])), 2) / (2**(num_genes//2)-1) * (learning_rate_range[1] - learning_rate_range[0])
    num_hidden_neurons = num_hidden_neurons_range[0] + int(''.join(map(str, chromosome[num_genes//2:])), 2) / (2**(num_genes//2)-1) * (num_hidden_neurons_range[1] - num_hidden_neurons_range[0])
    return learning_rate, int(num_hidden_neurons)

def fitness(learning_rate, num_hidden_neurons, X_train, y_train, X_val, y_val, input_size, output_size):
    model = SimpleNN(input_size= input_size, hidden_size=num_hidden_neurons, output_size=output_size)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    num_epochs = 2
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        outputs = torch.sigmoid(model(X_val))
        predictions = []
        for output in outputs:
            if output < 0.5:
                predictions.append(0)
            else:
                predictions.append(1)
        predictions = torch.tensor(predictions)
        accuracy = (predictions == y_val.view(-1)).sum().item() / len(y_val.view(-1))

    return accuracy


In [4]:
def genetic_algorithm(population_size, num_genes, learning_rate_range, num_hidden_neurons_range, X_train, y_train, X_val, y_val, num_generations, input_size, output_size):
    population = initialize_population(population_size, num_genes)
    max_score = -100
    for generation in range(num_generations):
        fitness_scores = []
        for chromosome in population:
            learning_rate, num_hidden_neurons = decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range, num_genes)
            curr_fitness = fitness(learning_rate, num_hidden_neurons, X_train, y_train, X_val, y_val, input_size, output_size)
            fitness_scores.append(curr_fitness)
            if curr_fitness > max_score:
                max_score = curr_fitness

        print(f"gen: {generation} | top 2: {sorted(fitness_scores)[-2:]}", end=" ")
        # parents based on fitness
        best_scores = sorted(fitness_scores)[-2:]
        parents = population[np.argsort(fitness_scores)[-2:]]
        print("| parents", parents)


        # Crossover
        crossover_point = np.random.randint(1, num_genes)
        children = []
        for i in range(population_size - 2):
            parent1 = parents[i % 2]
            parent2 = parents[(i + 1) % 2]

            child = []
            for j in range(num_genes):
                child.append([parent1[j], parent2[j]][np.random.randint(0,2)])

            # child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child = np.asarray(child)
            children.append(child)

        # Mutation
        mutation_rate = 0.1
        for i in range(population_size - 2):
            if np.random.rand() < mutation_rate:
                mutation_point = np.random.randint(num_genes)
                children[i][mutation_point] = 1 - children[i][mutation_point]
        best_chromosome = population[np.argmax(fitness_scores)]
        # Update population
        population = np.vstack((parents, np.array(children)))

    best_learning_rate, best_num_hidden_neurons = decode_chromosome(best_chromosome, learning_rate_range, num_hidden_neurons_range, num_genes)

    return best_learning_rate, best_num_hidden_neurons, max_score


In [5]:
heart_ds = pd.read_csv("./Heart Attack.csv")

In [6]:
heart_ds['class'] = heart_ds['class'].apply(lambda x: 0 if x == "negative" else 1)
heart_ds

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,64,1,66,160,83,160.0,1.80,0.012,0
1,21,1,94,98,46,296.0,6.75,1.060,1
2,55,1,64,160,77,270.0,1.99,0.003,0
3,64,1,70,120,55,270.0,13.87,0.122,1
4,55,1,64,112,65,300.0,1.08,0.003,0
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,0
1315,66,1,84,125,55,149.0,1.33,0.172,1
1316,45,1,85,168,104,96.0,1.24,4.250,1
1317,54,1,58,117,68,443.0,5.80,0.359,1


In [7]:
heart_x = heart_ds[["age", "gender", "impluse", "pressurehight","pressurelow" ,"glucose", "kcm", "troponin"]]
heart_y = heart_ds["class"]
heart_x, heart_y = torch.tensor(heart_x.values, dtype=torch.float32), torch.tensor(heart_y.values, dtype=torch.float32).view(-1,1)
heart_x_train, heart_x_val, heart_y_train, heart_y_val = train_test_split(heart_x, heart_y, test_size=0.2, random_state=42)
# heart_x_train, heart_x_val, heart_y_train, heart_y_val

In [8]:
learning_rate_range = [0.001, 0.8]
num_hidden_neurons_range = [5, 100]
input_size, output_size = 8, 1

best_learning_rate, best_num_hidden_neurons, xyz = genetic_algorithm(population_size=50, num_genes=16,
                                                                learning_rate_range=learning_rate_range,
                                                                num_hidden_neurons_range=num_hidden_neurons_range,
                                                                X_train=heart_x_train, y_train=heart_y_train,
                                                                X_val=heart_x_val, y_val=heart_y_val,
                                                                num_generations=100, input_size = input_size, output_size=output_size)

print("Best learning rate:", best_learning_rate)
print("Best number of hidden neurons:", best_num_hidden_neurons)
print("Best fitness score:", xyz)


gen: 0 | top 2: [0.6212121212121212, 0.6325757575757576] | parents [[1 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0]
 [0 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0]]
gen: 1 | top 2: [0.6287878787878788, 0.6553030303030303] | parents [[0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0]
 [0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 0]]
gen: 2 | top 2: [0.6174242424242424, 0.6174242424242424] | parents [[0 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0]
 [0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0]]
gen: 3 | top 2: [0.6363636363636364, 0.6704545454545454] | parents [[0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0]
 [0 0 1 0 0 1 1 0 0 1 0 1 1 1 1 0]]
gen: 4 | top 2: [0.6212121212121212, 0.625] | parents [[0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0]
 [0 0 1 0 0 1 1 0 0 1 0 1 1 1 1 0]]
gen: 5 | top 2: [0.6174242424242424, 0.625] | parents [[0 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0]
 [0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0]]
gen: 6 | top 2: [0.6212121212121212, 0.625] | parents [[0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0]
 [0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0]]
gen: 7 | top 2: [0.6287878787878788, 0.6363636363636364] | parents [[0 0 